## A custom Model
This notebook shows how to implement and train a model other than the baseline used for this challenge.

In [1]:
import os
import sys
sys.path.insert(1, os.path.realpath(os.path.pardir))


import torch

from utils import creating_dataset


### Load data

In [2]:
DATA_PATH = r"/msc/home/vsharm64/projects/BCI_Kaggle/dataset_v2_blocks/dataset_v2_blocks"
data_paths = dict(datasets=[DATA_PATH],
                    hand_type = ['left', 'right'], # [left, 'right']
                    human_type = ['health', 'amputant'], # [amputant, 'health']
                    test_dataset_list = ['fedya_tropin_standart_elbow_left'])
data_config = creating_dataset.DataConfig(**data_paths)
train_dataset, test_dataset = creating_dataset.get_datasets(data_config, )

Getting val datasets
Number of moves: 72 | Dataset: fedya_tropin_standart_elbow_left
Reorder this dataset fedya_tropin_standart_elbow_left True
Getting train datasets
Number of moves: 72 | Dataset: fedya_tropin_standart_elbow_left
Reorder this dataset fedya_tropin_standart_elbow_left True
Number of moves: 70 | Dataset: valery_first_standart_elbow_left
Reorder this dataset valery_first_standart_elbow_left True
Number of moves: 135 | Dataset: alex_kovalev_standart_elbow_left
Reorder this dataset alex_kovalev_standart_elbow_left True
Number of moves: 72 | Dataset: anna_makarova_standart_elbow_left
Reorder this dataset anna_makarova_standart_elbow_left True
Number of moves: 62 | Dataset: artem_snailbox_standart_elbow_left
Reorder this dataset artem_snailbox_standart_elbow_left True
Number of moves: 144 | Dataset: matthew_antonov_standart_elbow_left
Reorder this dataset matthew_antonov_standart_elbow_left True
Number of moves: 144 | Dataset: misha_korobok_standart_elbow_left
Reorder this da

In [3]:
# model.decoder_embedding

In [4]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import tqdm

# class ConditionalTransformer(nn.Module):
#     def __init__(self, input_dim, output_dim, d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, dropout=0.1):
#         super(ConditionalTransformer, self).__init__()
#         self.input_dim = input_dim
#         self.output_dim = output_dim
#         self.d_model = d_model

#         self.encoder_embedding = nn.Linear(input_dim, d_model)
#         self.decoder_embedding = nn.Linear(output_dim, d_model)
#         self.positional_encoding = nn.Parameter(torch.zeros(1, 5000, d_model))  # Positional encoding

#         self.transformer = nn.Transformer(d_model=d_model, nhead=nhead, num_encoder_layers=num_encoder_layers, 
#                                           num_decoder_layers=num_decoder_layers, dim_feedforward=dim_feedforward, dropout=dropout)

#         self.fc_out = nn.Linear(d_model, output_dim)
#         self.downsample = nn.Sequential(
#             nn.Linear(output_dim, output_dim // 2),
#             nn.ReLU(),
#             nn.Linear(output_dim // 2, output_dim)
#         )

#     def forward(self, src, tgt):
#         src_seq_len, tgt_seq_len = src.shape[1], tgt.shape[1]
#         print(src.shape)
#         src_emb = self.encoder_embedding(src) + self.positional_encoding[:, :src_seq_len, :]
#         print(tgt.shape)
#         tgt_emb = self.decoder_embedding(tgt) + self.positional_encoding[:, :tgt_seq_len, :]

#         src_emb = src_emb.permute(1, 0, 2)  # (seq_len, batch_size, d_model)
#         tgt_emb = tgt_emb.permute(1, 0, 2)  # (seq_len, batch_size, d_model)

#         output = self.transformer(src_emb, tgt_emb)
#         output = self.fc_out(output.permute(1, 0, 2))  # (batch_size, seq_len, output_dim)

#         return self.downsample(output)

# # Hyperparameters
# input_dim = 8
# output_dim = 20
# d_model = 256
# nhead = 8
# num_encoder_layers = 4
# num_decoder_layers = 4
# dim_feedforward = 512
# dropout = 0.1

# # Initialize the model, criterion, and optimizer
# model = ConditionalTransformer(input_dim, output_dim, d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, dropout)
# criterion = nn.MSELoss()
# optimizer = optim.Adam(model.parameters(), lr=1e-3)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# n_epochs = 25

# # Dummy train_dataset - replace with actual dataset
# # Example shapes from your data
# X_shape = (8, 256)
# Y_shape = (20, 32)

# # Generate dummy data for demonstration
# # train_dataset = [(np.random.randn(*X_shape), np.random.randn(*Y_shape)) for _ in range(100)]

# for epoch in range(n_epochs):
#     model.train()
#     epoch_loss = 0
#     for X, Y in tqdm.tqdm(train_dataset):
#         X, Y = torch.tensor(X.T).to(device), torch.tensor(Y).to(device)  # Transpose X to match expected input shape
#         tgt = torch.zeros(Y.shape[0], Y.shape[1], dtype=torch.float32).to(device)  # Dummy target input for the transformer decoder
#         print(tgt.shape)
        
#         optimizer.zero_grad()
        
#         Y_hat = model(X.unsqueeze(0), tgt.unsqueeze(0)).squeeze(0)  # Add batch dimension
#         loss = criterion(Y_hat, Y)
#         loss.backward()
#         optimizer.step()
        
#         epoch_loss += loss.item()

#     print(f"Epoch {epoch+1} Loss: {epoch_loss:.4f}")


## Define model
We define a simple Multi Layer Perceptron, fully connected feedforward neural network with 2 hidden layers. The model is implemented using PyTorch.

In [6]:
class MLP(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(input_size, hidden_size),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        return self.layers(x)

Try making a prediction using this model.

In [7]:

X, Y = train_dataset[0]
print(f"X shape: {X.shape}, Y shape: {Y.shape}")

n_inputs, n_outputs = X.shape[0], Y.shape[0]
n_hidden = 64

model = MLP(n_inputs, n_hidden, n_outputs)


Y_hat = model(torch.tensor(X.T)).detach().numpy().T
print(f"Predictions shape: {Y_hat.shape}")

assert Y.shape == Y_hat.shape, "Predictions have the wrong shape!"

X shape: (8, 256), Y shape: (20, 32)
Predictions shape: (20, 256)


AssertionError: Predictions have the wrong shape!

As upi cam see. The model's predictions are of the wrong shape: the outputs are not downsampled! Remember that inputs are sampled at 200Hz, but outputs are meant to be at 25Hz. We need to downsample the outputs to match the expected shape.

You can choose your own downsampling method, but make sure that your predictions are aligned to the targets!

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from numpy.linalg import solve

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class NTK:
    def __init__(self, reg=1, sigma=1.0, alpha=0.5):
        super().__init__()
        self.reg = reg
        self.sigma = sigma
        self.alpha = alpha
        self.sol = None
        self.Xtrain = None

    def ntk_kernel(self, pair1, pair2):
        out = pair1 @ pair2.T + 1
        N1 = np.sum(np.power(pair1, 2), axis=-1).reshape(-1, 1) + 1
        N2 = np.sum(np.power(pair2, 2), axis=-1).reshape(-1, 1) + 1

        XX = np.sqrt(N1 @ N2.T)
        out = out / XX

        out = np.clip(out, -1, 1)

        first = (
            1
            / np.pi
            * (out * (np.pi - np.arccos(out)) + np.sqrt(1.0 - np.power(out, 2)))
            * XX
        )
        sec = 1 / np.pi * out * (np.pi - np.arccos(out)) * XX
        out = first + sec

        C = 1
        return out / C

    def gaussian_kernel(self, pair1, pair2):
        sq_dist = np.sum(pair1**2, axis=1).reshape(-1, 1) + np.sum(pair2**2, axis=1) - 2 * np.dot(pair1, pair2.T)
        return np.exp(-sq_dist / (2 * self.sigma**2))

    def kernel(self, pair1, pair2):
        ntk = self.ntk_kernel(pair1, pair2)
        gaussian = self.gaussian_kernel(pair1, pair2)
        return self.alpha * ntk + (1 - self.alpha) * gaussian

    def fit(self, Xtrain, ytrain):
        K = self.kernel(Xtrain, Xtrain)
        sol = solve(K + self.reg * np.eye(len(K)), ytrain).T
        self.sol = sol
        self.Xtrain = Xtrain
        return self

    def predict(self, X):
        K = self.kernel(self.Xtrain, X)
        return (self.sol @ K).T


In [16]:
simple_downsample(Y).shape

(20, 4)

In [17]:
Y.shape

(20, 32)

In [18]:
X.T.shape

(256, 8)

In [11]:
import numpy as np

def simple_downsample(x: np.ndarray) -> np.ndarray:
    return x[:, ::data_config.down_sample_target]

# Y_hat = model(torch.tensor(X.T)).detach().numpy().T
# Y_hat = simple_downsample(Y_hat)


# assert Y.shape == Y_hat.shape, "Predictions have the wrong shape!"

model1 = NTK(reg=0.01,alpha=1.0)
model1.fit(X.T, simple_downsample(Y.T))

# model1.predict(X.T)
# Fit model and predict
# model1 = model1.fit(torch.Tensor(X.T), train_out1)


ValueError: solve: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (m,m),(m,n)->(m,n) (size 32 is different from 256)

In [5]:
import numpy as np

def simple_downsample(x: np.ndarray) -> np.ndarray:
    return x[:, ::data_config.down_sample_target]

Y_hat = model(torch.tensor(X.T)).detach().numpy().T
Y_hat = simple_downsample(Y_hat)


assert Y.shape == Y_hat.shape, "Predictions have the wrong shape!"

For simplicity, you can build the downsampling into your model

In [6]:
class MLP(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(input_size, hidden_size),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        x = self.layers(x)
        return x[::data_config.down_sample_target, :]
    

model = MLP(n_inputs, n_hidden, n_outputs)

Y_hat = model(torch.tensor(X.T)).detach().numpy().T
assert Y.shape == Y_hat.shape, "Predictions have the wrong shape!"

## Training

We define a very simple training procedure.

In [ ]:
n_epochs = 25
lr = 1e-3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MLP(n_inputs, n_hidden, n_outputs).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = torch.nn.MSELoss()

for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    
    epoch_loss = 0
    for X,Y in train_dataset:
        X, Y = torch.tensor(X.T).to(device), torch.tensor(Y).to(device)
        Y_hat = model(X).T

        loss = criterion(Y_hat, Y)
        loss.backward()
        epoch_loss += loss.item()

        optimizer.step()

    print(f"Epoch {epoch+1} Loss: {epoch_loss:.4f}")

    

Epoch 1 Loss: 20065223.0544
Epoch 2 Loss: 556769311.3171
Epoch 3 Loss: 584892621.1726
Epoch 4 Loss: 104045801.0032


### Let's plot some predictions

In [ ]:
import matplotlib.pyplot as plt


X, Y = test_dataset[0]

Y_hat = model(torch.tensor(X.T).to(device)).detach().cpu().numpy().T

f, axes = plt.subplots(5, 1, figsize=(10, 10), sharex=True)

for i, ax in enumerate(axes):
    ax.plot(Y[i], label="True", lw=2, color='k')
    ax.plot(Y_hat[i], label="Predicted", lw=1, color='r')

    ax.legend()
